<a href="https://colab.research.google.com/github/Souhib-khalbous/Drug-Response-Prediction/blob/main/DRP_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drug Response Prediction (DRP) Project**

In [ ]:
#Mount Google Drive when you are using Cloab

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Load datasets
#Drug Response datasets
gdsc1 = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Drug Response Datasets/GDSC (Sanger)/Drug_sensitivity_IC50_(Sanger_GDSC1)_subsetted.csv')
gdsc2 = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Drug Response Datasets/GDSC (Sanger)/Drug_sensitivity_IC50_(Sanger_GDSC2)_subsetted.csv')
prism = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Drug Response Datasets/Repurposing (Braod)/PRISM_Repurposing_Public_24Q2_subsetted.csv')

#Omics datasets
copy_number = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Omics Datasets/WES (Broad)/Copy_Number_Public_24Q2_subsetted.csv')
mutations = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Omics Datasets/WES (Broad)/Damaging_Mutations_subsetted.csv')
expression = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Omics Datasets/RNA (Broad)/Expression_Public_24Q2_subsetted.csv')
proteomics = pd.read_csv('/content/drive/MyDrive/Drug Response Prediction (DRP) Project/DRP Project/Datasets/Omics Datasets/RPPA (CCLE)/Proteomics_subsetted.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1. Normalization** (Mapping between 0 and 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Function for normalization
def normalize_data(df, exclude_index):
    scaler = MinMaxScaler()
    df_numeric = df.drop(df.columns[exclude_index], axis=1)
    df_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)
    df_scaled.insert(exclude_index, df.columns[exclude_index], df.iloc[:, exclude_index])
    return df_scaled

# Columns to exclude from normalization (index-based since there's no header)
exclude_index = 0

# Apply normalization
gdsc1_normalized = normalize_data(gdsc1, exclude_index)
gdsc2_normalized = normalize_data(gdsc2, exclude_index)
prism_normalized = normalize_data(prism, exclude_index)
copy_number_normalized = normalize_data(copy_number, exclude_index)
mutations_normalized = normalize_data(mutations, exclude_index)
expression_normalized = normalize_data(expression, exclude_index)
proteomics_normalized = normalize_data(proteomics, exclude_index)

# Save normalized datasets (optional, if you want to keep a copy of them)
gdsc1_normalized.to_csv('gdsc1_normalized.csv', index=False)
gdsc2_normalized.to_csv('gdsc2_normalized.csv', index=False)
prism_normalized.to_csv('prism_normalized.csv', index=False)
copy_number_normalized.to_csv('copy_number_normalized.csv', index=False)
mutations_normalized.to_csv('mutations_normalized.csv', index=False)
expression_normalized.to_csv('expression_normalized.csv', index=False)
proteomics_normalized.to_csv('proteomics_normalized.csv', index=False)

# Display the first few rows of one of the normalized datasets for verification
gdsc1_normalized.head()


,Unnamed: 0,ERLOTINIB (GDSC1:1),AICA RIBONUCLEOTIDE (GDSC1:1001),VELBAN (GDSC1:1004),CIS-DDP (GDSC1:1005),CYTARABINE (GDSC1:1006),DOCETAXEL (GDSC1:1007),ABITREXATE (GDSC1:1008),ATRA (GDSC1:1009),GEFITINIB (GDSC1:1010),...,BMS-509744 (GDSC1:63),CMK (GDSC1:64),CHLORIDINE (GDSC1:71),JW-7-52-1 (GDSC1:83),A-443654 (GDSC1:86),GW843682X (GDSC1:87),ENTINOSTAT (GDSC1:88),PARTHENOLIDE (GDSC1:89),LLL CPD (GDSC1:9),TGX 221 (GDSC1:94)
0,ACH-000001,NaN,NaN,0.229534,NaN,NaN,0.835794,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACH-000002,NaN,0.736845,0.436190,0.955283,NaN,0.854276,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.583986,NaN,0.798300,NaN
2,ACH-000004,NaN,0.910335,0.852758,0.342196,0.244331,NaN,0.287521,NaN,NaN,...,NaN,NaN,0.817683,0.279065,0.828553,0.376468,0.392994,0.758733,0.597483,NaN
3,ACH-000006,NaN,0.801090,0.670702,NaN,NaN,NaN,0.322785,NaN,NaN,...,0.960545,NaN,NaN,0.825123,0.475869,0.522104,0.471594,NaN,NaN,NaN
4,ACH-000007,NaN,NaN,NaN,NaN,0.738887,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.994881,NaN


# **2. Missing Values?**

Let's deal with them by imputing missing values with the ***mean*** of the non-missing values

In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Function to impute missing values with the mean
def impute_missing_values(df, exclude_index):
    df_numeric = df.drop(df.columns[exclude_index], axis=1)
    imputer = SimpleImputer(strategy='mean')
    df_imputed = pd.DataFrame(imputer.fit_transform(df_numeric), columns=df_numeric.columns)
    df_imputed.insert(exclude_index, df.columns[exclude_index], df.iloc[:, exclude_index])
    return df_imputed

# Columns to exclude from imputation (index-based since there's no header)
exclude_index = 0

# Apply mean imputation
gdsc1_imputed = impute_missing_values(gdsc1_normalized, exclude_index)
gdsc2_imputed = impute_missing_values(gdsc2_normalized, exclude_index)
prism_imputed = impute_missing_values(prism_normalized, exclude_index)
copy_number_imputed = impute_missing_values(copy_number_normalized, exclude_index)
mutations_imputed = impute_missing_values(mutations_normalized, exclude_index)
expression_imputed = impute_missing_values(expression_normalized, exclude_index)
proteomics_imputed = impute_missing_values(proteomics_normalized, exclude_index)

# Save imputed datasets (optional, if you want to keep a copy of them)
gdsc1_imputed.to_csv('gdsc1_imputed.csv', index=False)
gdsc2_imputed.to_csv('gdsc2_imputed.csv', index=False)
prism_imputed.to_csv('prism_imputed.csv', index=False)
copy_number_imputed.to_csv('copy_number_imputed.csv', index=False)
mutations_imputed.to_csv('mutations_imputed.csv', index=False)
expression_imputed.to_csv('expression_imputed.csv', index=False)
proteomics_imputed.to_csv('proteomics_imputed.csv', index=False)

# Display the first few rows of one of the imputed datasets for verification
gdsc1_imputed.head()


,Unnamed: 0,ERLOTINIB (GDSC1:1),AICA RIBONUCLEOTIDE (GDSC1:1001),VELBAN (GDSC1:1004),CIS-DDP (GDSC1:1005),CYTARABINE (GDSC1:1006),DOCETAXEL (GDSC1:1007),ABITREXATE (GDSC1:1008),ATRA (GDSC1:1009),GEFITINIB (GDSC1:1010),...,BMS-509744 (GDSC1:63),CMK (GDSC1:64),CHLORIDINE (GDSC1:71),JW-7-52-1 (GDSC1:83),A-443654 (GDSC1:86),GW843682X (GDSC1:87),ENTINOSTAT (GDSC1:88),PARTHENOLIDE (GDSC1:89),LLL CPD (GDSC1:9),TGX 221 (GDSC1:94)
0,ACH-000001,0.529706,0.817984,0.229534,0.781005,0.719569,0.835794,0.526583,0.548684,0.68434,...,0.762272,0.797502,0.629935,0.430727,0.652047,0.450155,0.676996,0.828244,0.734626,0.5
1,ACH-000002,0.529706,0.736845,0.436190,0.955283,0.719569,0.854276,0.526583,0.548684,0.68434,...,0.762272,0.797502,0.629935,0.430727,0.652047,0.450155,0.583986,0.828244,0.798300,0.5
2,ACH-000004,0.529706,0.910335,0.852758,0.342196,0.244331,0.691031,0.287521,0.548684,0.68434,...,0.762272,0.797502,0.817683,0.279065,0.828553,0.376468,0.392994,0.758733,0.597483,0.5
3,ACH-000006,0.529706,0.801090,0.670702,0.781005,0.719569,0.691031,0.322785,0.548684,0.68434,...,0.960545,0.797502,0.629935,0.825123,0.475869,0.522104,0.471594,0.828244,0.734626,0.5
4,ACH-000007,0.529706,0.817984,0.511837,0.781005,0.738887,0.691031,0.526583,0.548684,0.68434,...,0.762272,0.797502,0.629935,1.000000,0.652047,0.450155,0.676996,0.828244,0.994881,0.5


# **3. Integrate Drug Response Data**

In [ ]:
# Import necessary libraries
import pandas as pd

# Print the number of columns for each dataset before integration
print(f'Number of columns in GDSC1 dataset: {gdsc1_imputed.shape[1]}')
print(f'Number of columns in GDSC2 dataset: {gdsc2_imputed.shape[1]}')
print(f'Number of columns in PRISM dataset: {prism_imputed.shape[1]}')

# Merge the datasets on the cell line ID
drug_response_merged = pd.merge(gdsc1_imputed, gdsc2_imputed, on='Unnamed: 0', suffixes=('_gdsc1', '_gdsc2'))
drug_response_merged = pd.merge(drug_response_merged, prism_imputed, on='Unnamed: 0')

# Print the number of columns after initial merge
print(f'Number of columns after merging GDSC1, GDSC2, and PRISM datasets: {drug_response_merged.shape[1]}')

# Resolve any conflicts or redundancies
# Here we will average the duplicated columns if they exist. Adjust this step as per your conflict resolution strategy.
# Example: If there are duplicated columns, average their values
for column in drug_response_merged.columns:
    if '_gdsc1' in column or '_gdsc2' in column:
        base_column_name = column.split('_')[0]
        if base_column_name in drug_response_merged.columns:
            drug_response_merged[base_column_name] = drug_response_merged[[column, base_column_name]].mean(axis=1)
            drug_response_merged.drop(columns=[column], inplace=True)

# Print the number of columns after resolving conflicts
print(f'Number of columns after resolving conflicts: {drug_response_merged.shape[1]}')

# Rename the index column to cell_line_id for clarity
drug_response_merged.rename(columns={'Unnamed: 0': 'cell_line_id'}, inplace=True)

# Save the unified drug response dataset
drug_response_merged.to_csv('drug_response_merged.csv', index=False)

# Display the first few rows of the unified drug response dataset for verification
drug_response_merged.head()


Number of columns in GDSC1 dataset: 311
Number of columns in GDSC2 dataset: 154
Number of columns in PRISM dataset: 6766
Number of columns after merging GDSC1, GDSC2, and PRISM datasets: 7229
Number of columns after resolving conflicts: 7229


,cell_line_id,ERLOTINIB (GDSC1:1),AICA RIBONUCLEOTIDE (GDSC1:1001),VELBAN (GDSC1:1004),CIS-DDP (GDSC1:1005),CYTARABINE (GDSC1:1006),DOCETAXEL (GDSC1:1007),ABITREXATE (GDSC1:1008),ATRA (GDSC1:1009),GEFITINIB (GDSC1:1010),...,TALC (BRD:BRD-M92352362-002-02-3),EFONIDIPINE-MONOETHANOLATE (BRD:BRD-M92675308-003-07-1),LASOFOXIFENE (BRD:BRD-M93148412-001-02-6),DICHLOROACETATE (BRD:BRD-M97302542-001-03-6),CEVIMELINE (BRD:BRD-M98279124-300-01-1),COLESEVALAM (BRD:BRD-U08520523-000-01-0),TYLOXAPOL (BRD:BRD-U25960968-000-01-9),SEVELAMER (BRD:BRD-U45393375-000-01-6),TYLOXAPOL (BRD:BRD-U48018661-000-01-9),PHOSPHATIDYLCHOLINE (BRD:BRD-U51753822-000-01-1)
0,ACH-000001,0.529706,0.817984,0.229534,0.781005,0.719569,0.835794,0.526583,0.548684,0.68434,...,0.393923,0.484027,0.613124,0.562324,0.360123,0.680057,0.515772,0.623282,0.503481,0.423244
1,ACH-000002,0.529706,0.736845,0.436190,0.955283,0.719569,0.854276,0.526583,0.548684,0.68434,...,0.579515,0.479663,0.767642,0.491018,0.561992,0.596852,0.461004,0.499382,0.567831,0.454895
2,ACH-000004,0.529706,0.910335,0.852758,0.342196,0.244331,0.691031,0.287521,0.548684,0.68434,...,0.579515,0.479663,0.557208,0.491018,0.561992,0.596852,0.461004,0.499382,0.567831,0.454895
3,ACH-000006,0.529706,0.801090,0.670702,0.781005,0.719569,0.691031,0.322785,0.548684,0.68434,...,0.579515,0.479663,0.945104,0.491018,0.561992,0.596852,0.461004,0.499382,0.567831,0.454895
4,ACH-000007,0.529706,0.817984,0.511837,0.781005,0.738887,0.691031,0.526583,0.548684,0.68434,...,0.478067,0.629305,0.363595,0.437351,0.683675,0.569303,0.458859,0.313475,0.761277,0.381444


# **4. Integrate Omics Data**

In [ ]:
# Import necessary libraries
import pandas as pd


# Print the number of columns for each dataset before integration
print(f'Number of columns in Copy Number dataset: {copy_number_imputed.shape[1]}')
print(f'Number of columns in Mutations dataset: {mutations_imputed.shape[1]}')
print(f'Number of columns in Expression dataset: {expression_imputed.shape[1]}')
print(f'Number of columns in Proteomics dataset: {proteomics_imputed.shape[1]}')

# Merge the datasets on the cell line ID
omics_merged = pd.merge(copy_number_imputed, mutations_imputed, on='Unnamed: 0', suffixes=('_copy_number', '_mutations'))
omics_merged = pd.merge(omics_merged, expression_imputed, on='Unnamed: 0')
omics_merged = pd.merge(omics_merged, proteomics_imputed, on='Unnamed: 0')

# Print the number of columns after initial merge
print(f'Number of columns after merging Copy Number, Mutations, Expression, and Proteomics datasets: {omics_merged.shape[1]}')

# Resolve any conflicts or redundancies if necessary
# (Here we assume there are no conflicts to resolve)

# Rename the index column to cell_line_id for clarity
omics_merged.rename(columns={'Unnamed: 0': 'cell_line_id'}, inplace=True)

# Save the unified omics dataset
omics_merged.to_csv('omics_merged.csv', index=False)

# Display the first few rows of the unified omics dataset for verification
omics_merged.head()


Number of columns in Copy Number dataset: 23780
Number of columns in Mutations dataset: 17680
Number of columns in Expression dataset: 19154
Number of columns in Proteomics dataset: 12756
Number of columns after merging Copy Number, Mutations, Expression, and Proteomics datasets: 73367


,cell_line_id,LINC02593,TTC34_copy_number,MIR4251,LINC01134,MIR4252,LINC01783,MIR3675,MIR1290,MIR1256,...,ZBTB47 tr|H7BXD3|H7BXD3_HUMAN'],MYO6 tr|Q5JVM0|Q5JVM0_HUMAN'],ZNF66 tr|I3L2D0|I3L2D0_HUMAN'],MRPS17 tr|I3L0E3|I3L0E3_HUMAN'],SNX10 sp|Q9Y5X0|SNX10_HUMAN'],EID1 sp|Q9Y6B2|EID1_HUMAN'],SPCS1 sp|Q9Y6A9|SPCS1_HUMAN'],IER3IP1 sp|Q9Y5U9|IR3IP_HUMAN'],DOCK9 tr|H0Y572|H0Y572_HUMAN'],MRPL42 sp|Q9Y6G3|RM42_HUMAN']
0,ACH-000967,0.114633,0.034801,0.066996,0.105034,0.310608,0.352667,0.204437,0.289434,0.303566,...,0.268375,0.606630,0.624281,0.784927,0.492167,0.459399,0.552681,0.516516,0.551832,0.400423
1,ACH-000919,0.111964,0.027108,0.062539,0.100397,0.292975,0.337095,0.180916,0.271607,0.283442,...,0.268375,0.641536,0.624281,0.825457,0.500877,0.294678,0.447203,0.511031,0.471757,1.000000
2,ACH-000832,0.115059,0.071417,0.064863,0.102815,0.302171,0.345217,0.139779,0.308161,0.324706,...,0.268375,0.722386,0.624281,0.884293,0.492167,0.459399,0.598928,0.511031,0.551832,0.641096
3,ACH-000866,0.117878,0.030300,0.065074,0.103035,0.303006,0.367382,0.755778,0.285446,0.299064,...,0.268375,0.795746,0.624281,0.706403,0.492167,0.459399,0.766215,0.377576,0.551832,0.416438
4,ACH-000652,0.120689,0.076845,0.070330,0.108504,0.323798,0.364316,0.088908,0.084964,0.072747,...,0.204640,0.947485,0.624281,0.769499,0.492167,0.459399,0.630413,0.511031,0.680005,0.509734
